In [53]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.cluster import MiniBatchKMeans

from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import svm 

from catboost import CatBoostRegressor
from sklearn import linear_model
import math

from scipy.optimize import minimize
from sklearn.metrics import mean_squared_log_error

In [54]:
MODE='test'

In [55]:
if MODE=='validation':
    filename='train.csv'
elif MODE=='test':
    filename='Xtest.csv'

In [56]:
data = pd.read_csv(filename, 
                   header=0, 
                   sep=',',
                   encoding = "windows-1251", 
                   error_bad_lines=False)
                    

In [57]:
X=pd.DataFrame(data['Password'])
X=X['Password'].apply(str)

In [58]:
if MODE=='validation':
    y=data['Times']
    y=y.apply(int)

In [59]:
def rmsle(y_pred, y_test) : 
    assert len(y_test) == len(y_pred)
    return np.sqrt(np.mean((np.log(1+y_pred) - np.log(1+y_test))**2))

In [62]:
def getSymbols(words):
    symbols=set()
    for q in range(words.shape[0]):
        for w in range(len(words[q])):
            symbols.add(words[q][w])
    return np.array(list(symbols))


In [63]:
def get_count(word):
    count=0
    for i_s in range(len(symbs)):
        if(symbs[i_s] in word):
            count+=1
    return count

In [64]:
def getAllSuffixes(words, times):
    suffixes={}
    suffixescounts={}
    for word in words:
        for q in range(len(word)):
            tempword=str(word[q])
            for w in range(q+1, len(word)):
                tempword+=word[w]
                if suffixes.get(tempword)==None:
                    suffixes[tempword]=times[q]
                    suffixescounts[tempword]=1
                else:
                    suffixes[tempword]+=times[q]
                    suffixescounts[tempword]+=1

            
    return [suffixes, suffixescounts]

In [65]:
def get_Thresh_complete(times_thresh ,df):
    primery_th=df[(df.Times >times_thresh)]
    primery_th_pss= list(primery_th['Password'].apply(str))
    primery_th_times= list(primery_th['Times'].apply(int))
    
    suffixes= getAllSuffixes(primery_th_pss, primery_th_times)
    return suffixes

FEATURE GENERATION

In [66]:
if MODE=='validation':
    alphabet= ["a", "b", "c", "d", 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z' ]
    alphabetUp= ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z' ]

    vowelUp= ['A', 'E', 'I', 'O', 'U']
    vowel= ['a', 'e', 'i', 'o', 'u']

    digits=['0','1','2','3','4','5','6','7','8','9']

    consonant=[ "b", "c", "d", 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 'p', 'q', 'r', 's', 't', 'v', 'w', 'x', 'y', 'z' ]
    consonantUp=['B',"C", 'D', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'X', 'Y', 'Z' ]

    allsym = getSymbols(X.values.reshape(-1))
    othersym= list(set(allsym) - set(alphabet) - set(alphabetUp))

In [67]:
if MODE=='validation':
    timesthresh=100
    freq=45


In [68]:
if MODE=='validation':
    thresh = get_Thresh_complete(timesthresh,data)
    val=list(thresh[0].values())
    cts=list(thresh[1].values())
    keys=list(thresh[0].keys())

    for q in range(len(val)):
        val[q]=val[q]/cts[q]
    tempdf= pd.DataFrame()
    tempdf['val']= val
    tempdf['cts']= cts
    tempdf['keys']=keys
    
    tempdf= tempdf[(tempdf['cts']>= freq)]
    val=tempdf['val'].values
    cts=tempdf['cts'].values
    
    keys=tempdf['keys'].values

In [69]:
tempdf.shape

(140, 3)

In [70]:
df= pd.DataFrame()

In [71]:
df['firstvowel']= X.apply(lambda x: x[0] in vowel )
df['firstvowelUp']= X.apply(lambda x: x[0] in vowelUp )

In [72]:
df['firstcons']= X.apply(lambda x: x[0] in consonant )
df['firstconsUp']= X.apply(lambda x: x[0] in consonantUp )

In [73]:
df['firstdigit']= X.apply(lambda x: x[0] in digits )
df['firstother']= X.apply(lambda x: x[0] in othersym )

df['lastdigit']= X.apply(lambda x: x[-1] in digits )
df['lastother']= X.apply(lambda x: x[-1] in othersym )

In [74]:
df['fupper']=X.apply(lambda x: x[0].isupper())
df['lupper']=X.apply(lambda x: x[-1].isupper())

In [75]:
df['isdigit']=X.apply(lambda x: x.isdigit())

In [76]:
df['islower']=X.apply(lambda x: x.islower())

In [77]:
df['fupper']=X.apply(lambda x: x.isupper())

In [78]:
for q in range(len(allsym)):
    df[str(q)+'symbol']=X.apply(lambda x : allsym[q] in x)
    if q%10==0:
        print(q)
    

0
10
20
30
40
50
60
70
80
90


In [79]:
binary=list(df.columns)

In [80]:
df['cnt']=X.apply(lambda x: len(x))

In [81]:
symbs=vowel
df['conscnt']=X.apply(get_count)

In [82]:
symbs=consonant
df['vowcnt']=X.apply(get_count)

In [83]:
symbs=consonantUp
df['consUpcnt']=X.apply(get_count)

In [84]:
symbs=vowelUp
df['vowUpcnt']=X.apply(get_count)

In [85]:
symbs=digits
df['digitscnt']=X.apply(get_count)

In [86]:
symbs=othersym
df['othersymcnt']=X.apply(get_count)

In [87]:
numeric= list(set(df.columns) - set(binary))

KEYS FEATURES GENERATION

In [88]:
%%time
for q in range(len(keys)):
    df[str(q) + '_key']= X.apply(lambda x: keys[q] in x )
    if q%50==0:
        print(q)

0
50
100
CPU times: user 35.1 s, sys: 79.9 ms, total: 35.1 s
Wall time: 34.8 s


INIT MODELS

In [41]:
models=[
    RandomForestRegressor(max_depth=2,verbose=1, n_estimators=100, n_jobs=-1),
    linear_model.SGDRegressor(max_iter=2000, tol=0.1, verbose=1),
    GradientBoostingRegressor(verbose=1),
    CatBoostRegressor(iterations=500,learning_rate=0.1, depth=3)
]

CHOOSE MODEL NUMBER

In [51]:
index= 1
model=models[index]

TRAIN

In [47]:
MODE='validation_full'

In [48]:
if MODE=='validation':
    X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=_test_size, random_state=42)
elif MODE=='validation_full':
    X_train=df
    y_train=y

In [ ]:
MODE

In [ ]:
if MODE=='validation' or 'validation_full':
    model.fit(X_train, np.log(y_train +1))

  


In [52]:
for q in range(0, len(X_train)-100000,100000):
    model.partial_fit(X_train[q: q+100000], np.log(y_train[q: q+100000] +1))

-- Epoch 1
Norm: 0.53, NNZs: 244, Bias: 0.743287, T: 100000, Avg. loss: 0.080971
Total training time: 0.09 seconds.
-- Epoch 1
Norm: 0.53, NNZs: 247, Bias: 0.782036, T: 100000, Avg. loss: 0.078251
Total training time: 0.09 seconds.
-- Epoch 1
Norm: 0.53, NNZs: 248, Bias: 0.804723, T: 100000, Avg. loss: 0.080575
Total training time: 0.09 seconds.
-- Epoch 1
Norm: 0.53, NNZs: 250, Bias: 0.815090, T: 100000, Avg. loss: 0.080062
Total training time: 0.09 seconds.
-- Epoch 1
Norm: 0.54, NNZs: 250, Bias: 0.827435, T: 100000, Avg. loss: 0.080509
Total training time: 0.09 seconds.
-- Epoch 1
Norm: 0.55, NNZs: 250, Bias: 0.835380, T: 100000, Avg. loss: 0.080367
Total training time: 0.09 seconds.
-- Epoch 1
Norm: 0.55, NNZs: 251, Bias: 0.834223, T: 100000, Avg. loss: 0.080908
Total training time: 0.09 seconds.
-- Epoch 1
Norm: 0.55, NNZs: 251, Bias: 0.834224, T: 100000, Avg. loss: 0.081252
Total training time: 0.09 seconds.
-- Epoch 1
Norm: 0.54, NNZs: 251, Bias: 0.830369, T: 100000, Avg. loss: 

PREDICT

In [94]:
if MODE=='validation':
    preds= np.exp(model.predict(X_test))-1
    rmsle(y_test.values, preds)
elif MODE== 'test':
    preds= np.exp(model.predict(df))-1

In [ ]:
dftemp = {'Id' : range(df.shape[0]), 'Times': preds}

In [ ]:
if MODE=='test':

    dfconst.to_csv('Answers_140_feats.csv' ,encoding = "utf8", index=False)

COSNTANT SUBMIT

In [ ]:
minfunc = minimize(rmsle,x0=1,method='BFGS', tol=1e-3)

In [ ]:
Passes=np.array([minfunc.x[0]]*data['Id'].shape[0])
dftemp = {'Id' : range(Passes.shape[0]), 'Times': Passes}
dfconst=pd.DataFrame(dftemp)

In [ ]:
dfconst.to_csv('constAns.csv' ,encoding = "utf8", index=False)

In [ ]:
dfconst.shape